# Description - Create SWARM File for Power Spectrum generation for each ROI

This notebook will do the following operations:

* Compute the spectrograms for each run separately using the Welch method. This part is done in parallel via swarm jobs. The main output for each job is: ```/data/SFIMJGC_HCP7T/HCP7T/${SBJ}/${RUN}/${RUN}.Signal.V4_grp.welch.pkl```


***
# Import Libraries

In [1]:
import os
import os.path as osp
import numpy as np
import pandas as pd
from utils.variables import Resources_Dir, DATA_DIR
from utils.basics import get_available_runs

from scipy.stats import kruskal, wilcoxon, ttest_ind

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Load Name of Runs on each group (Awake, Drowsy)

In [2]:
Manuscript_Runs = get_available_runs(when='final')
print('++ INFO: Number of Runs = %d' % len(Manuscript_Runs))

++ INFO: Number of Runs = 404


In [3]:
Drowsy_Runs = get_available_runs(when='final',type='drowsy')
Awake_Runs  = get_available_runs(when='final',type='awake')
print('++ INFO: Number of Awake  Runs = %d' % len(Awake_Runs))
print('++ INFO: Number of Drowsy Runs = %d' % len(Drowsy_Runs))

++ INFO: Number of Awake  Runs = 210
++ INFO: Number of Drowsy Runs = 194


# Generate Swarm Infrastructure

These jobs are very fast, less than 5 minutes per run

In [5]:
if not osp.exists('./N10_mPP_Peridiogram_FullRun.logs'):
    os.mkdir('./N10_mPP_Peridiogram_FullRun.logs')

In [6]:
# Create Swarm file for extracting representative power
# =====================================================
os.system('echo "#swarm -f ./N10_mPP_Peridiogram_FullRun.SWARM.sh -g 16 -t 16 -b 30 --partition quick,norm --time 00:5:00 --logdir ./N10_mPP_Peridiogram_FullRun.logs" > ./N10_mPP_Peridiogram_FullRun.SWARM.sh')
for sbj_run in Manuscript_Runs:
    sbj,run  = sbj_run.split('_',1)
    out_dir  = osp.join(DATA_DIR,sbj,run)
    for region in ['V4_grp','V4lt_grp','V4ut_grp']:
        os.system('echo "export SBJ={sbj} RUN={run} REGION={reg} DATADIR={ddir}; sh ./N10_mPP_Peridiogram_FullRun.sh" >> ./N10_mPP_Peridiogram_FullRun.SWARM.sh'.format(sbj=sbj, run=run, reg=region, ddir=DATA_DIR))

# Submit Jobs to Cluster

Login to biowulf, and run the following commands on a terminal:

```bash
cd /data/SFIMJGC_HCP7T/hcp7t_fv_sleep/Notebooks
rm ./N10_mPP_Peridiogram_FullRun.logs/*
swarm -f ./N10_mPP_Peridiogram_FullRun.SWARM.sh -g 16 -t 16 --partition quick,norm --time 00:20:00 --logdir ./N10_mPP_Peridiogram_FullRun.logs
watch -n 120 squeue -u javiergc
```

***

# Test all outputs have been generated by swarm jobs

In [7]:
num_files=0
suffix='mPP'
region='V4ut_grp'
for sbj_run in Manuscript_Runs:
    sbj,run  = sbj_run.split('_',1)
    out_file = osp.join(DATA_DIR,sbj,run,'{run}_{suffix}.Signal.{region}.welch.pkl'.format(run=run, region=region, suffix=suffix))
    if not osp.exists(out_file):
        print('++ WARNING: File missing [%s]' % out_file)
    else:
        num_files +=1
print('++ INFO: Number of available files = %d' % num_files)

++ INFO: Number of available files = 404
